### Reading the data

In [333]:
from Data import * 

In [94]:
import torch
with open(path, 'r', encoding="utf8") as f:
    ids = torch.LongTensor(tokens)
    token = 0
    for line in f:
        words = line.split()
        for word in words:
            ids[token] = dictionary.word2idx[word]
            token += 1

In [112]:
train_corpus = ids

In [114]:
def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 10

In [230]:
batch_size = 1
train_data = batchify(train_corpus, batch_size)

In [312]:
vocab_size = len(dictionary)
embedding_size = hidden_size = 32

model = LSTM_Model(vocab_size, embedding_size, hidden_size)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [233]:
seq_size = 1
def get_batch(source, i):
    seq_len = min(seq_size, len(source) - 1 - i)
    
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

In [245]:
def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

In [313]:
lr = 10
log_interval = 10
epoch = 1
seq_size = 100

def train():
    # Turn on training mode which enables dropout.
    model.train()
    total_loss = 0.
    #start_time = time.time()
    ntokens = len(dictionary)
    
    hidden = model.init_hidden(batch_size)
    
    
    for batch, i in enumerate(range(0, train_data.size(0) - 1, seq_size)):
        data, targets = get_batch(train_data, i)
        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        
        #print(hidden.shape)
        hidden = repackage_hidden(hidden)
        #print(hidden[0].shape)
        
        
        
        model.zero_grad()
        output, hidden = model(data, hidden)
        
        
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        optimizer.step()
        

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        
        
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            #elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // seq_size, lr,\
                    #elapsed * 1000 / args.log_interval, 
                        cur_loss, math.exp(cur_loss)))
            total_loss = 0
            #start_time = time.time()

In [314]:
import math
train()

| epoch   1 |    10/18003 batches | lr 10.00 | loss 10.51 | ppl 36855.89
| epoch   1 |    20/18003 batches | lr 10.00 | loss  8.76 | ppl  6376.45
| epoch   1 |    30/18003 batches | lr 10.00 | loss  8.06 | ppl  3166.28
| epoch   1 |    40/18003 batches | lr 10.00 | loss  7.80 | ppl  2436.17
| epoch   1 |    50/18003 batches | lr 10.00 | loss  7.30 | ppl  1481.30
| epoch   1 |    60/18003 batches | lr 10.00 | loss  7.22 | ppl  1370.95
| epoch   1 |    70/18003 batches | lr 10.00 | loss  7.47 | ppl  1750.42
| epoch   1 |    80/18003 batches | lr 10.00 | loss  7.44 | ppl  1699.23
| epoch   1 |    90/18003 batches | lr 10.00 | loss  7.11 | ppl  1223.72
| epoch   1 |   100/18003 batches | lr 10.00 | loss  6.68 | ppl   792.44
| epoch   1 |   110/18003 batches | lr 10.00 | loss  7.51 | ppl  1822.74
| epoch   1 |   120/18003 batches | lr 10.00 | loss  7.37 | ppl  1583.55
| epoch   1 |   130/18003 batches | lr 10.00 | loss  7.30 | ppl  1475.48
| epoch   1 |   140/18003 batches | lr 10.00 | loss

| epoch   1 |  1140/18003 batches | lr 10.00 | loss  6.42 | ppl   614.47
| epoch   1 |  1150/18003 batches | lr 10.00 | loss  5.93 | ppl   375.14
| epoch   1 |  1160/18003 batches | lr 10.00 | loss  6.34 | ppl   566.62
| epoch   1 |  1170/18003 batches | lr 10.00 | loss  6.72 | ppl   826.42
| epoch   1 |  1180/18003 batches | lr 10.00 | loss  6.53 | ppl   683.58
| epoch   1 |  1190/18003 batches | lr 10.00 | loss  6.72 | ppl   825.06
| epoch   1 |  1200/18003 batches | lr 10.00 | loss  6.33 | ppl   562.66
| epoch   1 |  1210/18003 batches | lr 10.00 | loss  6.31 | ppl   552.69
| epoch   1 |  1220/18003 batches | lr 10.00 | loss  6.02 | ppl   412.07
| epoch   1 |  1230/18003 batches | lr 10.00 | loss  6.21 | ppl   499.77
| epoch   1 |  1240/18003 batches | lr 10.00 | loss  6.39 | ppl   594.41
| epoch   1 |  1250/18003 batches | lr 10.00 | loss  7.00 | ppl  1096.93
| epoch   1 |  1260/18003 batches | lr 10.00 | loss  6.64 | ppl   763.02
| epoch   1 |  1270/18003 batches | lr 10.00 | loss

KeyboardInterrupt: 